# Keras 101: A simple Neural Network for House Pricing regression


In this post, we will be covering some basics of data exploration and building a model with Keras in order to help us on predicting the selling price of a given house in the Boston (MA) area. As an application of this model in the real world, you can think about being a real state agent looking for a tool to help you on your day-to-day duties, which for me, at least, sounds pretty good when compared to just gut-estimation.

For this exercise, we will be using the [Plotly](https://plot.ly/python/) library instead of the good ol' fashioned matplotlib, due to having more interactive plots, which for sure help in understanding the data. We will also use the [Scikit-Learn](https://scikit-learn.org/stable/) and [Keras](https://keras.io/) for building the models, [Pandas](https://pandas.pydata.org/) library to manipulate our data and the [SHAP library](https://github.com/slundberg/shap) to generate explanations for our trained model.

### Importing the dataset

In this example, we wil be using the sklearn.datasets module, which contains the Boston dataset. You could also use the keras.datasets module, but this one does not contain the labels of the features, so we decided to use scikits one. Let's also convert it to a Pandas DataFrame and print it's head.




In [54]:
from sklearn.datasets import load_boston
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.models import model_from_json

boston_dataset = load_boston()

df = pd.DataFrame(boston_dataset.data, columns=boston_dataset.feature_names)
df['MEDV'] = boston_dataset.target
df = df.iloc[:250,:]
df.head(n=10)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning:

Function load_boston is deprecated; `load_boston` is deprecated in 1.0 and will be removed in 1.2.

    The Boston housing prices dataset has an ethical problem. You can refer to
    the documentation of this function for further details.

    The scikit-learn maintainers therefore strongly discourage the use of this
    dataset unless the purpose of the code is to study and educate about
    ethical issues in data science and machine learning.

    In this special case, you can fetch the dataset from the original
    source::

        import pandas as pd
        import numpy as np


        data_url = "http://lib.stat.cmu.edu/datasets/boston"
        raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
        data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
        target = raw_df.values[1::2, 2]

    Alternative datasets include the California housing dataset (i.e

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2
5,0.02985,0.0,2.18,0.0,0.458,6.430,58.7,6.0622,3.0,222.0,18.7,394.12,5.21,28.7
6,0.08829,12.5,7.87,0.0,0.524,6.012,66.6,5.5605,5.0,311.0,15.2,395.60,12.43,22.9
7,0.14455,12.5,7.87,0.0,0.524,6.172,96.1,5.9505,5.0,311.0,15.2,396.90,19.15,27.1
8,0.21124,12.5,7.87,0.0,0.524,5.631,100.0,6.0821,5.0,311.0,15.2,386.63,29.93,16.5
9,0.17004,12.5,7.87,0.0,0.524,6.004,85.9,6.5921,5.0,311.0,15.2,386.71,17.10,18.9



### Exploratory Data Analysis

Making yourself comfortable and familiar with your dataset is a fundamental step to help you comprehend your data and draw better conclusions and explanations from your results.

Initially, let's plot a few box plots, which will help us to better visualizate anomalies and/or outliers in data distribution. If you are confused about what is a box plot and how it can help us to better visualizate the distribution of our data, here is a brief description from Ross (1977):

> In descriptive statistics, a box plot is a method for graphically depicting groups of numerical data through their quartiles. Box plots may also have lines extending vertically from the boxes (whiskers) indicating variability outside the upper and lower quartiles, hence the terms box-and-whisker plot and box-and-whisker diagram. Outliers may be plotted as individual points.


In [27]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import math

total_items = len(df.columns)
items_per_row = 3
total_rows = math.ceil(total_items / items_per_row)

fig = make_subplots(rows=total_rows, cols=items_per_row)

cur_row = 1
cur_col = 1

for index, column in enumerate(df.columns):
    fig.add_trace(go.Box(y=df[column], name=column), row=cur_row, col=cur_col)
    
    if cur_col % items_per_row == 0:
        cur_col = 1
        cur_row = cur_row + 1
    else:
        cur_col = cur_col + 1
    

fig.update_layout(height=1000, width=550,  showlegend=False)
fig.show()

These results do corroborate our initial assumptions about having outliers in some columns. Let's also plot some scatter plots for each feature and the target variable, as well as their intercept lines:


In [28]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import math
import numpy as np

total_items = len(df.columns)
items_per_row = 3
total_rows = math.ceil(total_items / items_per_row)

fig = make_subplots(rows=total_rows, cols=items_per_row, subplot_titles=df.columns)

cur_row = 1
cur_col = 1

for index, column in enumerate(df.columns):
    fig.add_trace(go.Scattergl(x=df[column], 
                            y=df['MEDV'], 
                            mode="markers", 
                            marker=dict(size=3)), 
                  row=cur_row, 
                  col=cur_col)
    
    intercept = np.poly1d(np.polyfit(df[column], df['MEDV'], 1))(np.unique(df[column]))
    
    fig.add_trace(go.Scatter(x=np.unique(df[column]), 
                             y=intercept, 
                             line=dict(color='red', width=1)), 
                  row=cur_row, 
                  col=cur_col)
    
    if cur_col % items_per_row == 0:
        cur_col = 1
        cur_row = cur_row + 1
    else:
        cur_col = cur_col + 1
    

fig.update_layout(height=1000, width=550, showlegend=False)
fig.show()


From this initial data exploration, we can have two major conclusions:

- There is a strong linear correlation between the RM (average number of rooms per dwelling) and LSTAT (% lower status of the population) with the target variable, being the RM a positive and the LSTAT a negative correlation.
- There are some records containing outliers, which we could preprocess in order to input our model with more normalized data.

### Data preprocessing

Before we proceed into any data preprocessing, it's important to split our data into training and test sets. We should not apply any kind of preprocessing into our data without taking into account that we should not leak information from our test set. For this step, we can use the *train_test_split* method from scikit-learn. In this case, we will use a split of 70% of the data for training and 30% for testing. We also set a random_state seed, in order to allow reprocibility.


In [29]:
from sklearn.model_selection import train_test_split

X = df.loc[:, df.columns != 'MEDV']
y = df.loc[:, df.columns == 'MEDV']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=123)

In order to provide a standardized input to our neural network, we need the perform the normalization of our dataset. This can be seen as an step to reduce the differences in scale that may arise from the existent features. We perform this normalization by subtracting the mean from our data and dividing it by the standard deviation. **One more time,  this normalization should only be performed by using the mean and standard deviation from the training set, in order to avoid any information leak from the test set.**


In [30]:
mean = X_train.mean(axis=0)
std = X_train.std(axis=0)

X_train = (X_train - mean) / std
X_test = (X_test - mean) / std


### Build our model

Due to the small amount of presented data in this dataset, we must be careful to not create an overly complex model, which could lead to overfitting our data. For this, we are going to adopt an architecture based on two Dense layers, the first with 128 and the second with 64 neurons, both using a ReLU activation function. A dense layer with a linear activation will be used as output layer.

In order to allow us to know if our model is properly learning, we will use a mean squared error loss function and to report the performance of it we will adopt the mean average error metric.

By using the summary method from Keras, we can see that we have a total of 5,121 parameters, which is acceptable for us.


In [15]:
from keras.models import Sequential
from keras.layers import Dense

model = Sequential()

model.add(Dense(128, input_shape=(13, ), activation='relu', name='dense_1'))
model.add(Dense(64, activation='relu', name='dense_2'))
model.add(Dense(1, activation='linear', name='dense_output'))

model.compile(optimizer='adam', loss='mse', metrics=['mae'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_1 (Dense)             (None, 128)               1792      
                                                                 
 dense_2 (Dense)             (None, 64)                8256      
                                                                 
 dense_output (Dense)        (None, 1)                 65        
                                                                 
Total params: 10,113
Trainable params: 10,113
Non-trainable params: 0
_________________________________________________________________



### Train our model

This step is pretty straightforward: fit our model with both our features and their labels, for a total amount of 100 epochs, separating 5% of the samples (18 records) as validation set.


In [16]:
history = model.fit(X_train, y_train, epochs=100, validation_split=0.05)

Epoch 1/100
6/6 [==============================] - 2s 44ms/step - loss: 630.7045 - mae: 23.6856 - val_loss: 549.3899 - val_mae: 22.1075
Epoch 2/100
6/6 [==============================] - 0s 7ms/step - loss: 591.0640 - mae: 22.8524 - val_loss: 511.7638 - val_mae: 21.2801
Epoch 3/100
6/6 [==============================] - 0s 7ms/step - loss: 547.6979 - mae: 21.9243 - val_loss: 471.1449 - val_mae: 20.3432
Epoch 4/100
6/6 [==============================] - 0s 7ms/step - loss: 501.6457 - mae: 20.8715 - val_loss: 425.5368 - val_mae: 19.2382
Epoch 5/100
6/6 [==============================] - 0s 7ms/step - loss: 446.6737 - mae: 19.5762 - val_loss: 374.4377 - val_mae: 17.9160
Epoch 6/100
6/6 [==============================] - 0s 7ms/step - loss: 388.1941 - mae: 18.0461 - val_loss: 316.8883 - val_mae: 16.3127
Epoch 7/100
6/6 [==============================] - 0s 9ms/step - loss: 324.8260 - mae: 16.3144 - val_loss: 257.4010 - val_mae: 14.4518
Epoch 8/100
6/6 [==============================] - 0s 

## Saving Architechture and Weights:

In [18]:
from keras.models import model_from_json
# serialize model to json
json_model = model.to_json()
#json_model
#save the model architecture to JSON file
with open('client1_nn.json', 'w') as json_file:
    json_file.write(json_model)

In [22]:
model.save_weights('client1_nn.h5')

In [23]:

weight_layer_1=model.layers[0].get_weights()[0]
bias_layer_1=model.layers[0].get_weights()[1]

weight_layer_2=model.layers[1].get_weights()[0]
bias_layer_2=model.layers[1].get_weights()[1]

weight_layer_3=model.layers[2].get_weights()[0]
bias_layer_3=model.layers[2].get_weights()[1]




In [ ]:
for layer in model.layers:
  print(layer.name)
  print("Weights")
  print("Shape: ",layer.get_weights()[0].shape,'\n',layer.get_weights()[0])
  print("Bias")
  print("Shape: ",layer.get_weights()[1].shape,'\n',layer.get_weights()[1],'\n')
  count+=1


In [25]:

print("Weights and biases of the layers before training the model: \n")
count=0
ls=[]
for layer in model.layers:
  print(layer.name)
  print("Weights")
  print("Shape: ",layer.get_weights()[0].shape,'\n',layer.get_weights()[0])
  print("Bias")
  print("Shape: ",layer.get_weights()[1].shape,'\n',layer.get_weights()[1],'\n')
  count+=1


Weights and biases of the layers before training the model: 

dense_1
Weights
Shape:  (13, 128) 
 [[-0.10810528 -0.04256306 -0.32970807 ... -0.2242326   0.06780985
  -0.2210991 ]
 [ 0.17021643  0.1209093  -0.13986905 ... -0.30583155  0.10146755
  -0.24131566]
 [ 0.05480978  0.10779033 -0.1265292  ...  0.03445881 -0.25715473
  -0.16110232]
 ...
 [ 0.14985943  0.07588912  0.05417326 ...  0.19791451 -0.04375117
  -0.02651396]
 [ 0.18616603  0.18403901  0.07211368 ... -0.01363186  0.03238907
   0.04770255]
 [ 0.14969552 -0.14312036  0.12826504 ... -0.1398269   0.05949314
   0.00341396]]
Bias
Shape:  (128,) 
 [ 0.10427078  0.03306334  0.14168441  0.09490094  0.10245746  0.11478715
  0.11075933  0.09493779  0.13837825  0.07687748  0.08022442  0.0880478
  0.02440797  0.10468347  0.09549816  0.14481948  0.03743838  0.07914388
  0.10685676  0.0871152   0.02271743  0.08798667  0.10973434  0.08685806
  0.03996862  0.10051718  0.12090113  0.10558893  0.11112651  0.09427129
  0.08655947  0.12190478

In [20]:
w,bias=model.layers[0].get_weights()
w=np.asarray(w)
w

array([[-0.10810528, -0.04256306, -0.32970807, ..., -0.2242326 ,
         0.06780985, -0.2210991 ],
       [ 0.17021643,  0.1209093 , -0.13986905, ..., -0.30583155,
         0.10146755, -0.24131566],
       [ 0.05480978,  0.10779033, -0.1265292 , ...,  0.03445881,
        -0.25715473, -0.16110232],
       ...,
       [ 0.14985943,  0.07588912,  0.05417326, ...,  0.19791451,
        -0.04375117, -0.02651396],
       [ 0.18616603,  0.18403901,  0.07211368, ..., -0.01363186,
         0.03238907,  0.04770255],
       [ 0.14969552, -0.14312036,  0.12826504, ..., -0.1398269 ,
         0.05949314,  0.00341396]], dtype=float32)

By plotting both loss and mean average error, we can see that our model was capable of learning patterns in our data without overfitting taking place (as shown by the validation set curves):


In [ ]:
fig = go.Figure()
fig.add_trace(go.Scattergl(y=history.history['loss'],
                    name='Train'))

fig.add_trace(go.Scattergl(y=history.history['val_loss'],
                    name='Valid'))


fig.update_layout(height=500, width=700,
                  xaxis_title='Epoch',
                  yaxis_title='Loss')
fig.show()


### Evaluate our model


In [26]:
mse_nn, mae_nn = model.evaluate(X_test, y_test)

print('Mean squared error on test data: ', mse_nn)
print('Mean absolute error on test data: ', mae_nn)

3/3 [==============================] - 0s 109ms/step - loss: 8.4515 - mae: 2.2009
Mean squared error on test data:  8.451534271240234
Mean absolute error on test data:  2.2009472846984863


# Retraining with updated model co-efficients:

## Loading the Previous Architecture of Model:


# Importing Model 1 weights:

In [32]:
import os
import tensorflow as tf
import keras

In [33]:
from keras.initializers import glorot_uniform

# Creating a new model from the saved JSON file
# reda the model from the JSOn file
with open('client1_nn.json', 'r') as json_file:
    json_savedModel= json_file.read()
#json_savedModel
#load the model architecture 
model_1 = tf.keras.models.model_from_json(json_savedModel)
model_1.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_1 (Dense)             (None, 128)               1792      
                                                                 
 dense_2 (Dense)             (None, 64)                8256      
                                                                 
 dense_output (Dense)        (None, 1)                 65        
                                                                 
Total params: 10,113
Trainable params: 10,113
Non-trainable params: 0
_________________________________________________________________


## Importing the List of numpy arrays of Updated Weights

In [35]:
import pickle
with open("updated_layer_1.json", "rb") as fp1:
  ls1=pickle.load(fp1)
with open("updated_layer_2.json", "rb") as fp2:
  ls2=pickle.load(fp2)
with open("updated_layer_3.json", "rb") as fp3:
  ls3=pickle.load(fp3)

In [36]:
print("Weight , Bias of layer 1")
np.shape(ls1[0]),np.shape(ls1[1])

Weight , Bias of layer 1


((13, 128), (128,))

In [37]:
print("Weight , Bias of layer 2")
np.shape(ls2[0]),np.shape(ls2[1])

Weight , Bias of layer 2


((128, 64), (64,))

In [38]:
print("Weight , Bias of layer 3")
np.shape(ls3[0]),np.shape(ls3[1])

Weight , Bias of layer 3


((64, 1), (1,))

## Setting the updated weights in model

In [39]:
import numpy as np

In [40]:
print(f" Layer 1 Weight {np.shape(ls2[0])}")

 Layer 1 Weight (128, 64)


In [41]:
model_1.layers[0].set_weights(ls1)

In [42]:
model_1.layers[1].set_weights(ls2)

In [43]:
model_1.layers[2].set_weights(ls3)

In [44]:
print("Round 1 of Averaging: Weights and biases of the layers before training : \n")
count=0
ls=[]
for layer in model_1.layers:
  print(layer.name)
  print("Weights")
  print("Shape: ",layer.get_weights()[0].shape,'\n',layer.get_weights()[0])
  print("Bias")
  print("Shape: ",layer.get_weights()[1].shape,'\n',layer.get_weights()[1],'\n')
  count+=1

Round 1 of Averaging: Weights and biases of the layers before training : 

dense_1
Weights
Shape:  (13, 128) 
 [[ 6.20943308e-02 -1.78483278e-01 -1.02627665e-01 ...  2.57962830e-02
  -7.57881925e-02 -1.13780670e-01]
 [-3.46610397e-02  6.03686534e-02 -1.22795708e-01 ...  1.10384226e-02
   1.25830933e-01 -3.39010730e-04]
 [ 9.89168361e-02 -1.88106611e-01  1.48603730e-02 ...  1.31532550e-04
  -2.36775130e-02 -1.26437113e-01]
 ...
 [ 3.86220217e-03  2.81191356e-02 -2.66750515e-01 ... -1.85343191e-01
   7.72679448e-02  9.53996703e-02]
 [-1.08798683e-01  1.21338507e-02  2.08816499e-01 ...  1.49667889e-01
   2.09382176e-02 -5.49083985e-02]
 [-5.45909479e-02  3.19619589e-02 -1.03366733e-01 ... -3.93435098e-02
   4.24711704e-02  1.92250177e-01]]
Bias
Shape:  (128,) 
 [0.05488506 0.08146667 0.12298182 0.12015612 0.07622987 0.10471848
 0.09643918 0.12511289 0.06834874 0.12589926 0.08692739 0.10243049
 0.14626242 0.07084406 0.07910706 0.09141494 0.02152244 0.08845909
 0.10820374 0.09974018 0.09458

In [45]:
client1_weight_layer_1=model_1.layers[0].get_weights()[0]
client1_bias_layer_1=model_1.layers[0].get_weights()[1]

client1_weight_layer_2=model_1.layers[1].get_weights()[0]
client1_bias_layer_2=model_1.layers[1].get_weights()[1]

client1_weight_layer_3=model_1.layers[2].get_weights()[0]
client1_bias_layer_3=model_1.layers[2].get_weights()[1]


# Now Retraing with the updated models on same data:


In [46]:
model_1.compile(optimizer='adam', loss='mse', metrics=['mae'])
model_1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_1 (Dense)             (None, 128)               1792      
                                                                 
 dense_2 (Dense)             (None, 64)                8256      
                                                                 
 dense_output (Dense)        (None, 1)                 65        
                                                                 
Total params: 10,113
Trainable params: 10,113
Non-trainable params: 0
_________________________________________________________________


In [47]:
history = model_1.fit(X_train, y_train, epochs=100, validation_split=0.05)

Epoch 1/100
6/6 [==============================] - 1s 39ms/step - loss: 165.7572 - mae: 12.1212 - val_loss: 109.5070 - val_mae: 10.0564
Epoch 2/100
6/6 [==============================] - 0s 8ms/step - loss: 111.7245 - mae: 9.8113 - val_loss: 66.3468 - val_mae: 7.8269
Epoch 3/100
6/6 [==============================] - 0s 7ms/step - loss: 67.9050 - mae: 7.3507 - val_loss: 34.6746 - val_mae: 5.5098
Epoch 4/100
6/6 [==============================] - 0s 8ms/step - loss: 38.3263 - mae: 5.0619 - val_loss: 16.6093 - val_mae: 3.4351
Epoch 5/100
6/6 [==============================] - 0s 7ms/step - loss: 23.6002 - mae: 3.7381 - val_loss: 9.2881 - val_mae: 2.4927
Epoch 6/100
6/6 [==============================] - 0s 7ms/step - loss: 19.3199 - mae: 3.4044 - val_loss: 7.3930 - val_mae: 2.1244
Epoch 7/100
6/6 [==============================] - 0s 8ms/step - loss: 17.4688 - mae: 3.3281 - val_loss: 6.4958 - val_mae: 1.7114
Epoch 8/100
6/6 [==============================] - 0s 7ms/step - loss: 15.2652 -

### Evaluate our model


In [48]:
mse_nn, mae_nn = model_1.evaluate(X_test, y_test)

print('Mean squared error on test data: ', mse_nn)
print('Mean absolute error on test data: ', mae_nn)

3/3 [==============================] - 0s 4ms/step - loss: 7.9996 - mae: 2.1831
Mean squared error on test data:  7.999634742736816
Mean absolute error on test data:  2.1830618381500244


# Automated Pipeline 

*   Step 1: Weights Initialization with count=0
*   Step 2: Obtaining Weights and Sending them for Averaging
*   Step 3: Waiting for untill we pull the averaged weights
*   Step 4: Setting the weights and fitting the model again
*   Step 5: Repeat 2 to 4 untill we are satissfied :)



In [49]:
def unpickling_weights(filename):
  with open(filename, "rb") as fp:
    ls=pickle.load(fp)
  return ls

In [84]:
ls1=unpickling_weights("updated_layer_1.json")
np.shape(ls3[0])

(64, 1)

In [52]:
rows,features=X_test.shape

In [62]:
json1="updated_layer_1.json"
json2="updated_layer_2.json"
json3="updated_layer_3.json"
model_name="client1_nn.json"

In [89]:
model,history,mse_nn,mae_nn,count=Training_evaluating(X_train,y_train,X_test,y_test,count,features=13,json1="updated_layer_1.json",json2="updated_layer_2.json",json3="updated_layer_3.json",model_name="client1_nn.json",weight_file_name="client1_nn.h5")

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_1 (Dense)             (None, 128)               1792      
                                                                 
 dense_2 (Dense)             (None, 64)                8256      
                                                                 
 dense_output (Dense)        (None, 1)                 65        
                                                                 
Total params: 10,113
Trainable params: 10,113
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
6/6 [==============================] - 1s 38ms/step - loss: 166.3592 - mae: 12.1154 - val_loss: 108.7497 - val_mae: 10.0384
Epoch 2/100
6/6 [==============================] - 0s 9ms/step - loss: 111.5673 - mae: 9.8004 - val_loss: 67.3409 - val_mae: 7.8928
Epoch 3/100
6/6 [=============================

In [85]:
print("Give value of count carefuully")

Give value of count carefuully


In [88]:
def Training_evaluating(X_train,y_train,X_test,y_test,count,features,json1=None,json2=None,json3=None,model_name=None,weight_file_name=None):

    if count==0:
      model = Sequential()
      model.add(Dense(128, input_shape=(features, ), activation='relu', name='dense_1'))
      model.add(Dense(64, activation='relu', name='dense_2'))
      model.add(Dense(1, activation='linear', name='dense_output'))
      model.compile(optimizer='adam', loss='mse', metrics=['mae'])
      model.summary()

      history = model.fit(X_train, y_train, epochs=100, validation_split=0.05)

      mse_nn, mae_nn = model.evaluate(X_test, y_test)

      print('Mean squared error on test data: ', mse_nn)
      print('Mean absolute error on test data: ', mae_nn)

      sending_models_and_weights(model,model_name,weight_file_name)
      count=1
    else:
      ls1=unpickling_weights(json1)      
      ls2=unpickling_weights(json2)      
      ls3=unpickling_weights(json3)

      # Creating a new model from the saved JSON file
      # read the model from the JSOn file
      with open(model_name, 'r') as json_file:
        json_savedModel= json_file.read()
      #json_savedModel
      #load the model architecture 
      model = tf.keras.models.model_from_json(json_savedModel)
      model.summary()
      model.layers[0].set_weights(ls1)
      model.layers[1].set_weights(ls2)
      model.layers[2].set_weights(ls3)   
      
      model.compile(optimizer='adam', loss='mse', metrics=['mae'])
      history = model.fit(X_train, y_train, epochs=100, validation_split=0.05)

      mse_nn, mae_nn = model.evaluate(X_test, y_test)

      print('Mean squared error on test data: ', mse_nn)
      print('Mean absolute error on test data: ', mae_nn)
      count+=1
      
    return model,history,mse_nn,mae_nn,count

def sending_models_and_weights(model,model_name,weight_file_name):
    
   
    # serialize model to json
    json_model = model.to_json()
    #json_model
    #save the model architecture to JSON file
    with open(model_name, 'w') as json_file:
        json_file.write(json_model)

    model.save_weights(weight_file_name)
    

In [ ]:
def Training_evaluating(X_train,y_train,X_test,y_test,count,features,json1=None,json2=None,json3=None,model_name=None,weight_file_name=None):

    if count==0:
      model = Sequential()
      model.add(Dense(128, input_shape=(features, ), activation='relu', name='dense_1'))
      model.add(Dense(64, activation='relu', name='dense_2'))
      model.add(Dense(1, activation='linear', name='dense_output'))
      model.compile(optimizer='adam', loss='mse', metrics=['mae'])
      model.summary()

      history = model.fit(X_train, y_train, epochs=100, validation_split=0.05)

      mse_nn, mae_nn = model.evaluate(X_test, y_test)

      print('Mean squared error on test data: ', mse_nn)
      print('Mean absolute error on test data: ', mae_nn)

      sending_models_and_weights(model,model_name,weight_file_name)
      count=1
    else:
      ls1=unpickling_weights(json1)      
      ls2=unpickling_weights(json2)      
      ls3=unpickling_weights(json3)

      # Creating a new model from the saved JSON file
      # read the model from the JSOn file
      with open(model_name, 'r') as json_file:
        json_savedModel= json_file.read()
      #json_savedModel
      #load the model architecture 
      model_1 = tf.keras.models.model_from_json(json_savedModel)
      model_1.summary()
      model_1.layers[0].set_weights(ls1)
      model_1.layers[1].set_weights(ls2)
      model_1.layers[2].set_weights(ls3)   
      
      

        
      history = model_1.fit(X_train, y_train, epochs=100, validation_split=0.05)

      mse_nn, mae_nn = model_1.evaluate(X_test, y_test)

      print('Mean squared error on test data: ', mse_nn)
      print('Mean absolute error on test data: ', mae_nn)
      count+=1
      sending_models_and_weights(model_1,model_name,weight_file_name)
      
    return model_1,history,mse_nn,mae_nn,count

def sending_models_and_weights(model,model_name,weight_file_name):
    
   
    # serialize model to json
    json_model = model.to_json()
    #json_model
    #save the model architecture to JSON file
    with open(model_name, 'w') as json_file:
        json_file.write(json_model)

    model.save_weights(weight_file_name)
    

In [58]:
def func(a,b,cond=None):
  print(cond)
  return a+b
func(2,3,19)

19


5